In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.1MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix"

/content/drive/My Drive/Colab Notebooks/Matrix


In [7]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [8]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv' , low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model , x , y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [17]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [20]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [25]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features =  literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [33]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))
len(keys)

476

In [35]:
def get_name_feat(key):
  return 'feat_' + key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [37]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_lined', 'feat_case diameter', 'feat_nike product #',
       'feat_is energy star-certified', 'feat_watch band material',
       'feat_technician', 'feat_removable hood', 'feat_weight',
       'feat_guaranteed authentic', 'feat_name'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] *100

In [52]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [54]:
df[ df.brand != df.feat_brand].shape

(18228, 531)

In [59]:
df['brand'] = df['brand'].map(lambda x:str(x).lower() )
df[ df.brand == df.feat_brand].shape



(8846, 531)

In [62]:
feats = ['']

(-58.133398968282776, 4.206122611474276)

In [84]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_catalog',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_movement_cat',
 'feat_contains batteries_cat',
 'feat_finish_cat',
 'feat_material type_cat',
 'feat_lens type_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_lens material_cat',
 'feat_target audience_cat',
 'feat_bezel_cat',
 'feat_is waterproof_cat',
 'feat_frame style_cat',
 'feat_water-resistant_cat',
 'feat_airport friendly_cat',
 'feat_garment care_cat',
 'feat_manufacturer_cat',
 'feat_picture_cat',
 'feat_chest size_cat',
 'feat_case type:_cat',
 'feat_item color_cat',
 'feat_number of compartments_cat',
 'feat_spikes type_cat',
 'feat_item st

In [63]:

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model) 

(-57.36272058442563, 4.184892941569642)

In [0]:
feats = ['brand_cat' ,'feat_resizable_cat','feat_metal type_cat','feat_weight_cat' ,  'feat_brand_cat' ,  'feat_gender_cat','feat_material_cat', 'feat_style_cat' , 'feat_sport_cat' ]


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model) 

In [94]:
x = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(x,y);
eli5.show_weights(perm, feature_names=feats)

(-57.24402575576984, 4.1312255471516295)


Weight,Feature
0.2535 ± 0.0113,brand_cat
0.1029 ± 0.0052,feat_material_cat
0.0260 ± 0.0026,feat_gender_cat
0.0234 ± 0.0018,feat_resizable_cat
0.0183 ± 0.0015,feat_brand_cat
0.0130 ± 0.0008,feat_weight_cat
0.0082 ± 0.0016,feat_metal type_cat
0.0050 ± 0.0008,feat_style_cat
0.0000 ± 0.0000,feat_sport_cat


In [82]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/volt/dark grey', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '700155070', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/beige', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '366177018', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'main color': 'royal blue/white-bright blue', 'material': 'synthetic leather', 'gender': 'mens', 'country/region of manufacture': 'vietnam', 'country of manufacture': 'vietnam', 'type': 'cleats', 'condition': 'new with box'},
       {'style': 'basketball shoes', 'material': 'leather', 'country/region of manufacture': 'vietnam', 'condition': 'new with box'},
       {'style': 'athletic sneakers', 'country of manufacture': 'china', 'condition': 'new with box'}],
      dtype=object)